In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [ ]:
base_path = 'path/to/base-dir'

In [ ]:
reactome = pd.read_csv(f'{base_path}/Reactome.tsv', names=['genes']).squeeze().tolist()

In [ ]:
data = pd.read_csv(f'{base_path}/data/12_pvalues/pvalues.tsv', sep='\t', index_col=0)

In [ ]:
data = data.fillna(1)

In [ ]:
for method in data.columns:
    pvals = data[method].sort_values()

    y_true = np.isin(pvals.index.tolist(), reactome)
    precision = []
    recall = [0]
    for cutoff in range(0, len(pvals)):
        y_pred = [1 for _ in range(cutoff + 1)] + [0 for _ in range(len(pvals) - cutoff - 1)]

        (TN, FP), (FN, TP) = confusion_matrix(y_true=y_true, y_pred=y_pred)
        precision.append(TP / (TP + FP))
        recall.append(TP / (TP + FN))

    precision = [precision[0]] + precision + [sum(y_true) / len(y_true)]
    recall.append(1)
    pd.DataFrame.from_dict({'precision': precision, 'recall': recall}).to_csv(f'{base_path}/data/13_precision-recall/{method}.tsv', sep='\t', index=False)